# Dataset repacking implementation

Advance dataset operations, of sorting, offset, and length support

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-dataset"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/infctx-dev/notebook/trainer-v5-validation
TRAINER_DIR: /home/picocreator/rwkv-proj/infctx-dev/RWKV-v5
PROJECT_DIR: /home/picocreator/rwkv-proj/infctx-dev


In [5]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 2048 \
        --vocab_size world --skip-if-exists \
        "../model/L6-D2048-world-v5base-init.pth"

[2023-12-18 04:54:01,470] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.1'
---- Initializing model ----
No of layers: 6
Embedding size: 2048
Output model path: ../model/L6-D2048-world-v5base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


# Without dataset packing

In [33]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/test-dataset-repack-chunks.yaml"

Map (num_proc=16): 100%|██████████████| 310/310 [00:00<00:00, 798.15 examples/s]
Saving the dataset (1/1 shards): 100%|█| 310/310 [00:00<00:00, 6597.03 examples/
Saving the dataset (1/1 shards): 100%|███| 4/4 [00:00<00:00, 1833.98 examples/s]


In [34]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/test-dataset-repack-chunks.yaml" \
        --model.load_model="../model/L6-D2048-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v5-enwiki-10k-chunked/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Chunking 4096 - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=2 \
        --trainer.devices="{GPU_DEVICES}"

[2023-12-18 06:44:14,494] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/picocreator/rwkv-proj/infctx-dev/notebook/trainer-v5-validation/config/test-dataset-repack-chunks.yaml', '--model.load_model=../model/L6-D2048-world-v5base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-enwiki-10k-chunked/', '--trainer.logger.init_args.name=infctx-v5-dataset-packing - Chunking 4096 - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=2', '--trainer.devices=auto'], args=['fit', '-c', '/home/pi

# With dataset packing

In [31]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/test-dataset-repack.yaml"

Map (num_proc=16): 100%|███████████| 6409/6409 [00:00<00:00, 6571.76 examples/s]
Saving the dataset (1/1 shards): 100%|█| 323/323 [00:00<00:00, 6750.55 examples/
Saving the dataset (1/1 shards): 100%|█| 65/65 [00:00<00:00, 17764.37 examples/s


In [32]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/test-dataset-repack.yaml" \
        --model.load_model="../model/L6-D2048-world-v5base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v5-enwiki-10k-packing/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Packing - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=2 \
        --trainer.devices="{GPU_DEVICES}"

[2023-12-18 06:39:25,218] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/picocreator/rwkv-proj/infctx-dev/notebook/trainer-v5-validation/config/test-dataset-repack.yaml', '--model.load_model=../model/L6-D2048-world-v5base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v5-enwiki-10k-packing/', '--trainer.logger.init_args.name=infctx-v5-dataset-packing - Packing - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=2', '--trainer.devices=auto'], args=['fit', '-c', '/home/picocreator/rwk